In [ ]:
import requests
import ast
import csv
import pandas as pd
from IPython.display import clear_output

### Calendar Endpoint Collection

In [ ]:
import time
gameDates = []
for year in range(2015,2020):
    print(year)
    endpoint = f"http://data.nba.net/data/10s/prod/v1/{year}/schedule.json"
    schedule = requests.get(endpoint).json()
    for game in schedule['league']['standard']:
        gameDates.append(game['gameUrlCode'][0:8])
    time.sleep(1)

### Scoreboard Endpoint Collection
This contains all of the game IDs

## Get Game Stats

In [ ]:
def checkKey(dict, key): 
    if key in dict.keys(): 
        return True
    else:
        return False

In [ ]:
endpoint = 'https://data.nba.net/data/1h/prod/2017/teams_config.json'
teams_config = requests.get(endpoint).json()

teamIDs = []
x=0

timeOffset = dict([('ATL', -4),('BKN', -4),('BOS', -4),('CHA', -4),('CHI', -5),
                   ('CLE', -4),('DAL', -5),('DEN', -6),('DET', -4),('GSW', -7),
                   ('HOU', -5),('IND', -4),('LAC', -7),('LAL', -7),('MEM', -5),
                   ('MIA', -4),('MIL', -5),('MIN', -5),('NOP', -5),('NYK', -4),
                   ('OKC', -5),('ORL', -4),('PHI', -4),('PHX', -7),('POR', -7),
                   ('SAC', -7),('SAS', -5),('TOR', -4),('UTA', -6),('WAS', -4)])

for team in teams_config['teams']['config']:
    #print(team.keys())
    if(len(list(team.keys())) > 6):
        teamIDs.append(int(team['teamId']))
        x+=1
        #print(team['tricode'], team['ttsName'])
print(x)

In [ ]:
date = []
gameIdList = []
hTeamIds = []
vTeamIds = []
hTeamPrevWins = []
hTeamPrevLosses = []
vTeamPrevWins = []
vTeamPrevLosses = []
hTeamWin = []

for year in range(2015, 2019):
    print(year)
    endpoint = f"https://data.nba.net/data/10s/prod/v1/{year}/schedule.json"
    schedule = requests.get(endpoint).json()
    for game in schedule['league']['standard']:
        homeCode = game['gameUrlCode'][-6:-3]
        visCode = game['gameUrlCode'][-3:]
        if game['seasonStageId'] == 1:
            _ = -1
        elif not (homeCode in timeOffset.keys() and visCode in timeOffset.keys()):
            _ = -1
        elif int(game['hTeam']['score']) > int(game['vTeam']['score']) and (teamIDs.count(int(game['hTeam']['teamId']))>0 and teamIDs.count(int(game['vTeam']['teamId']))>0):
            date.append(game['gameUrlCode'][0:8])
            gameIdList.append(game['gameId'])
            hTeamIds.append(int(game['hTeam']['teamId']))
            vTeamIds.append(int(game['vTeam']['teamId']))
            hTeamPrevWins.append(int(game['hTeam']['win']) - 1)
            vTeamPrevLosses.append(int(game['vTeam']['loss']) - 1)
            vTeamPrevWins.append(int(game['vTeam']['win']))
            hTeamPrevLosses.append(int(game['hTeam']['loss']))
            hTeamWin.append(1)
        elif teamIDs.count(int(game['hTeam']['teamId']))>0 and teamIDs.count(int(game['vTeam']['teamId']))>0:
            date.append(game['gameUrlCode'][0:8])
            gameIdList.append(game['gameId'])
            hTeamIds.append(int(game['hTeam']['teamId']))
            vTeamIds.append(int(game['vTeam']['teamId']))
            vTeamPrevWins.append(int(game['vTeam']['win']) - 1)
            hTeamPrevLosses.append(int(game['hTeam']['loss']) - 1)
            hTeamPrevWins.append(int(game['hTeam']['win']))
            vTeamPrevLosses.append(int(game['vTeam']['loss']))
            hTeamWin.append(0)

In [ ]:
gamesDF = pd.DataFrame()
gamesDF['date'] = date
gamesDF['gameId'] = gameIdList
gamesDF['hTeamID'] = hTeamIds
gamesDF['hTeamWs'] = hTeamPrevWins
gamesDF['hTeamLs'] = hTeamPrevLosses
gamesDF['vTeamID'] = vTeamIds
gamesDF['vTeamWs'] = vTeamPrevWins
gamesDF['vTeamLs'] = vTeamPrevLosses
gamesDF['hWin'] = hTeamWin

gamesDF.head()

In [ ]:
columns = ['date', 'gameId', 'homeTeamID', 'homeWins', 'homeLosses', 'home_fastBreakPoints', 'home_pointsInPaint', 'home_biggestLead',
 'home_pointsOffTurnovers', 'home_points', 'home_fgm', 'home_fga', 'home_ftm',
 'home_fta', 'home_tpm', 'home_tpa', 'home_offReb', 'home_defReb', 'home_assists', 'home_pFouls', 'home_steals', 'home_turnovers',
 'home_blocks', 'home_plusMinus', 'visitorTeamID', 'visitorWins', 'visitorLosses', 'visitor_fastBreakPoints', 'visitor_pointsInPaint',
 'visitor_biggestLead', 'visitor_pointsOffTurnovers', 'visitor_points',
 'visitor_fgm', 'visitor_fga', 'visitor_tpa', 'visitor_offReb', 'visitor_defReb', 'visitor_assists', 'visitor_pFouls', 'visitor_steals',
 'visitor_turnovers', 'visitor_blocks', 'visitor_plusMinus']
df = pd.DataFrame(columns=columns)

df['date'], df['gameId'], df['homeTeamID'], df['homeWins'], df['homeLosses'] = gamesDF['date'], gamesDF['gameId'], gamesDF['hTeamID'], gamesDF['hTeamWs'], gamesDF['hTeamLs']
df['visitorTeamID'], df['visitorWins'], df['visitorLosses'] = gamesDF['vTeamID'], gamesDF['vTeamWs'], gamesDF['vTeamLs']
df['hWin'] = gamesDF['hWin']

for i, row in df.iterrows():
    thisDate = row['date']
    thisGameID = row['gameId']
    endpoint = f"https://data.nba.net/data/10s/prod/v1/{thisDate}/{thisGameID}_boxscore.json"
    boxScore = requests.get(endpoint).json()
    
    if 'stats' in boxScore.keys():
        boxScore = boxScore['stats']
        
        clear_output(wait=True)
        print(f'{round(100*(i/len(df)),2)}% complete')
        thisDate = row['date']
        thisGameID = row['gameId']
        endpoint = f"https://data.nba.net/data/10s/prod/v1/{thisDate}/{thisGameID}_boxscore.json"
        boxScore = requests.get(endpoint).json()['stats']
        for columnSet1 in columns[5:9]:
            df.at[i, columnSet1] = int(boxScore['hTeam'][columnSet1[5:]])
        for columnSet2 in columns[9:24]:
            df.at[i, columnSet2] = int(boxScore['hTeam']['totals'][columnSet2[5:]])
                
        for columnSet3 in columns[27:31]:
            df.at[i, columnSet3] = int(boxScore['vTeam'][columnSet3[8:]])
        for columnSet4 in columns[31:46]:
            df.at[i, columnSet4] = int(boxScore['vTeam']['totals'][columnSet4[8:]])

        
df.head()

In [ ]:
df.to_csv('gameData.csv')

In [ ]:
# Find out what team is each ID: 1610612744 1610612760 Kevin Durant 2015

endpoint = f"https://data.nba.net/data/10s/prod/v1/2015/teams.json"
teams = requests.get(endpoint).json()
for team in teams['league']['standard']:
    if team['teamId'] == "1610612739":
        print(team['city'], team['nickname'])

In [ ]:
teams = requests.get("http://data.nba.net/data/10s/prod/v1/2019/teams.json").json()
teamsDict = {}
for team in teams['league']['standard']:
    if team['isNBAFranchise']:
        teamsDict.update( {team['teamId'] : team['fullName'] })

In [ ]:
hTeam, hScore, vTeam, vScore = [], [], [], []

lastGameDate = 0
gameThisDay = 0
i = 0
for gameDate in gameDates:
    endpoint = f"https://data.nba.net/data/10s/prod/v1/{gameDate}/scoreboard.json"
    scoreboard = requests.get(endpoint).json()
    if lastGameDate == gameDate:
        gameThisDay += 1
    else:
        gameThisDay = 0
        lastGameDate = gameDate
    if scoreboard['games'][gameThisDay]['gameId'][0:3] != '001':
        teams = scoreboard['games'][gameThisDay]
        hTeam.append(teamsDict[teams['hTeam']['teamId']])
        vTeam.append(teamsDict[teams['vTeam']['teamId']])
        hScore.append(teams['hTeam']['score'])
        vScore.append(teams['vTeam']['score'])
        print('Home Team:', teamsDict[teams['hTeam']['teamId']], teams['hTeam']['score'], 'Visitor Team:', teamsDict[teams['vTeam']['teamId']], teams['vTeam']['score'])
    else:
        gameDates.remove(gameDate)
    i+=1

In [ ]:
# > 26 active players are in preseason games.....